In [1]:
"""
Classifying roof materials for building footprints from trained ResNet-18 models
Author: maxwell.cook@colorado.edu
"""

import sys, os, gc

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsat.models.classification import resnet18
from torch.nn.functional import softmax

# Custom functions (__functions.py)
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/earth-lab/opp-rooftop-mapping'
homedir = '/home/jovyan' # cyverse

# results_dir = os.path.join(maindir, 'results/resnet18/')
results_dir = os.path.join(homedir, 'results/resnet18-cv/')
print(results_dir)

print("Imports successful !")

/home/jovyan/results/resnet18-cv/
Imports successful !


In [2]:
# Best params from tuning
params = {'window_size': 78, 'batch_size': 64, 'learning_rate': 0.01, 'weight_decay': 0.01, 'momentum': 0.85, 'patience': 5}
print(f'Model params: {params}')

Model params: {'window_size': 78, 'batch_size': 64, 'learning_rate': 0.01, 'weight_decay': 0.01, 'momentum': 0.85, 'patience': 5}


In [ ]:
# Load MS building footprint data

In [5]:
fp = os.path.join(maindir, 'data/spatial/raw/dc_data/footprints/dc-ms_footprints.gpkg')
# fp = os.path.join(homedir, 'OPP/training/dc/dc-ms_footprints.gpkg')
footprints = gpd.read_file(fp)
footprints['uid'] = footprints.index + 1
footprints.head()

,release,capture_dates_range,geometry,uid
0,2,9/22/2020-9/23/2020,"POLYGON ((334794.168 4306846.311, 334799.662 4...",1
1,2,9/22/2020-9/23/2020,"POLYGON ((334703.572 4306870.743, 334701.114 4...",2
2,2,9/22/2020-9/23/2020,"POLYGON ((334666.123 4306432.288, 334671.751 4...",3
3,2,9/22/2020-9/23/2020,"POLYGON ((334616.324 4306162.858, 334621.692 4...",4
4,2,9/22/2020-9/23/2020,"POLYGON ((334622.665 4306603.184, 334630.936 4...",5


In [ ]:
# Create the roof image dataset for inference

In [ ]:
# Initialize the ResNet-18 model

In [ ]:
# Define whether to leverage cpu or gpu (for my local machine it is only cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get device for gpu or cpu
print(f'Using {device} for model eval ...')

In [ ]:
# Load the trained model for the current fold
best_fold = 4  # from holdout accuracy
model_fp = os.path.join(results_dir, f'denver-resnet18_fold{best_fold}.pth')
print(f"Loading model from path: {model_fp}")

checkpoint = torch.load(model_fp, map_location=device)

# Initialize the model architecture
model, _, _, _ = initialize_resnet_model(
    n_classes=n_classes,
    n_bands=n_bands,
    device=device,
    learning_rate=params['learning_rate'],
    momentum=params['momentum'],
    weight_decay=params['weight_decay'],
    patience=params['patience']
)

# Load the trained weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set the model to evaluation mode

In [ ]:
# Create a dataframe to store the results
n = dset.__len__() # length of the dataset
columns = ['tile', 'bbox', 'target', 'pred', 'conf']
res_df = pd.DataFrame(
    columns=columns,
    index=range(n)
)

In [ ]:
# Make inference on the footprint data
with torch.no_grad()
    for i, (data, target, tile, bbox) in enumerate(dloader):
        
        # move data to the same device as model
        data = data.float()
        model = model.float()
        
        output = model(data) 
        
        probs = torch.softmax(output, dim=1)
        conf, pred = torch.max(probs, 1)
        
        print(data.shape)
        print(pred)
    
        # store res to file
        ix = range(i * batch_size, min((i + 1) * batch_size, n))
        
        res_df.loc[ix, 'tile'] = tile
        res_df.loc[ix, 'bbox'] = bbox
        res_df.loc[ix, 'target'] = target
        res_df.loc[ix, 'pred'] = pred.cpu().numpy()
        res_df.loc[ix, 'conf'] = conf.cpu().numpy()

res_df.to_csv(results_file.archive_path)
res_df.head()

In [ ]:
# Make predictions on footprints in bbox